In [ ]:
import os
import json
import base64
from elasticsearch import Elasticsearch, helpers

from pprint import pprint

client = Elasticsearch(
    hosts=["http://localhost:9200"],
    api_key="SnppODJKVUJEUXJXcENRZDFndTA6LTJQclVnbllUNE9LLTJiUHJqSnBRZw==",
)

resp = client.ping()
print(resp)

def output(data):
    json_data = json.dumps(data.body, indent=4)
    print(json_data)



In [ ]:
resp = client.inference.put(
    task_type="text_embedding",
    inference_id="chunking-e5-model",
    inference_config={
        "service": "elasticsearch",
        "service_settings": {
            "num_allocations": 1,
            "num_threads": 1,
            "model_id": ".multilingual-e5-small"
        }
    },
)
print(resp)



In [ ]:

resp = client.indices.create(
    index="chunking-index",
    mappings={
        "properties": {
            "semantic_text": {
                "type": "semantic_text",
                "inference_id": "chunking-e5-model"
            },
            "attachment.content": {
                "type": "text",
                "copy_to": "semantic_text"
            }
        }
    },
)
print(resp)



In [ ]:
resp = client.ingest.put_pipeline(
    id="attachment",
    description="Extract PDF information",
    processors=[
        {
            "attachment": {
                "field": "data",
                "remove_binary": True
            }
        }
    ],
)
print(resp)



In [ ]:
pipeline_id = 'attachment'  # the pipeline we have created before
pdf_dir = 'WAC-BIG/'  # the folder with the documents 

# Function to convert PDF file to base64-encoded binary

def convert_pdf_to_base64(file_path):
    with open(file_path, "rb") as file:
        print(f'processing {file_path}...')
        return base64.b64encode(file.read()).decode('utf-8')

# Function to generate actions for the bulk API
def generate_actions(pdf_dir):
    for filename in os.listdir(pdf_dir):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(pdf_dir, filename)
            base64_encoded = convert_pdf_to_base64(file_path)
            yield {
                "_index": "chunking-index",
                "_source": {
                    "data": base64_encoded
                }
            }
       

# Use the helpers.bulk() function to index documents in bulk
helpers.bulk(client, generate_actions(pdf_dir), pipeline=pipeline_id, request_timeout=60*3)

print("Finished indexing PDF documents.")